In [1]:
import os
import csv
import pickle
import re

AUDIO_ROOT = 'C:/Users/tkddn/code/kaggle/train_audio'

def natural_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]


species_list = sorted(
    [d for d in os.listdir(AUDIO_ROOT) if os.path.isdir(os.path.join(AUDIO_ROOT, d))],
    key=natural_key
)

# 라벨 딕셔너리
label_dict = {species: idx for idx, species in enumerate(species_list)}

# 저장
with open('class_to_label.pkl', 'wb') as f:
    pickle.dump(label_dict, f)

with open('class_to_label.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['species', 'label'])
    for species, label in label_dict.items():
        writer.writerow([species, label])

print(label_dict)

{'21038': 0, '21116': 1, '21211': 2, '22333': 3, '22973': 4, '22976': 5, '24272': 6, '24292': 7, '24322': 8, '41663': 9, '41778': 10, '41970': 11, '42007': 12, '42087': 13, '42113': 14, '46010': 15, '47067': 16, '48124': 17, '50186': 18, '52884': 19, '64862': 20, '65336': 21, '65344': 22, '65349': 23, '65373': 24, '65419': 25, '65448': 26, '65547': 27, '65962': 28, '66016': 29, '66531': 30, '66578': 31, '66893': 32, '67082': 33, '67252': 34, '81930': 35, '126247': 36, '134933': 37, '135045': 38, '476537': 39, '476538': 40, '517119': 41, '523060': 42, '528041': 43, '548639': 44, '555086': 45, '555142': 46, '566513': 47, '714022': 48, '715170': 49, '787625': 50, '868458': 51, '963335': 52, '1139490': 53, '1192948': 54, '1194042': 55, '1346504': 56, '1462711': 57, '1462737': 58, '1564122': 59, 'amakin1': 60, 'amekes': 61, 'ampkin1': 62, 'anhing': 63, 'babwar': 64, 'bafibi1': 65, 'banana': 66, 'baymac': 67, 'bbwduc': 68, 'bicwre1': 69, 'bkcdon': 70, 'bkmtou1': 71, 'blbgra1': 72, 'blbwre1':

In [3]:
pip install efficientnet_pytorch

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ------------- -------------------------- 2.1/6.2 MB 11.8 MB/s eta 0:00:01
   ------------------------------ --------- 4.7/6.2 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 6.2/6.2 MB 11.2 MB/s eta 0:00:00
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16458 sha256=a2ed77315e4bf14b02cf7de7dbd77ded30c6592b46680587cdb39d8f810ee776
  Stored in directory: c:\users\tkddn\appdata\local\pip\cache\wheels\9c\3f\43\e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
Successfully built efficientnet_pytorch
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2
Note: you may need to restart the kern

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
from efficientnet_pytorch import EfficientNet

# ============================
# 1️⃣ 데이터 경로 및 설정
# ============================
DATA_DIR = 'C:/Users/tkddn/code/kaggle/train_audio_mel'  # 멜 스펙 저장된 폴더
BATCH_SIZE = 32
NUM_WORKERS = 4
NUM_EPOCHS = 10
LEARNING_RATE = 1e-4

# ============================
# 2️⃣ 데이터 전처리 및 데이터셋 준비
# ============================
transform = transforms.Compose([
    transforms.Resize(224),              # EfficientNet 기본 입력 크기
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet 평균
                         [0.229, 0.224, 0.225])  # ImageNet 표준편차
])

dataset = datasets.ImageFolder(root=DATA_DIR, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# ============================
# 3️⃣ EfficientNet 모델 생성 및 수정
# ============================
num_classes = len(dataset.classes)
model = EfficientNet.from_pretrained('efficientnet-b0')
in_features = model._fc.in_features
model._fc = nn.Linear(in_features, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# ============================
# 4️⃣ 손실함수, 옵티마이저, 스케줄러 정의
# ============================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# ============================
# 5️⃣ 학습 및 검증 함수 정의
# ============================
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total

def eval_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total

# ============================
# 6️⃣ 메인 학습 루프 실행
# ============================
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = eval_epoch(model, val_loader, criterion, device)
    scheduler.step()
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# ============================
# 7️⃣ 학습된 모델 저장
# ============================
torch.save(model.state_dict(), 'efficientnet_birdclef.pth')
print("✅ 모델 저장 완료! 경로: efficientnet_birdclef.pth")


FileNotFoundError: Found no valid file for the classes 1139490, 1192948, 1194042, 126247, 1346504, 134933, 135045, 1462711, 1462737, 1564122, 21038, 21116, 21211, 22333, 22973, 22976, 24272, 24292, 24322, 41663, 41778, 41970, 42007, 42087, 42113, 46010, 47067, 476537, 476538, 48124, 50186, 517119, 523060, 528041, 52884, 548639, 555086, 555142, 566513, 64862, 65336, 65344, 65349, 65373, 65419, 65448, 65547, 65962, 66016, 66531, 66578, 66893, 67082, 67252, 714022, 715170, 787625, 81930, 868458, 963335, amakin1, amekes, ampkin1, anhing, babwar, bafibi1, banana, baymac, bbwduc, bicwre1, bkcdon, bkmtou1, blbgra1, blbwre1, blcant4, blchaw1, blcjay1, blctit1, blhpar1, blkvul, bobfly1, bobher1, brtpar1, bubcur1, bubwre1, bucmot3, bugtan, butsal1, cargra1, cattyr, chbant1, chfmac1, cinbec1, cocher1, cocwoo1, colara1, colcha1, compau, compot1, cotfly1, crbtan1, crcwoo1, crebob1, cregua1, creoro1, eardov1, fotfly, gohman1, grasal4, grbhaw1, greani1, greegr, greibi1, grekis, grepot1, gretin1, grnk